<a href="https://colab.research.google.com/github/daehkim/RL-lecture/blob/master/lab6-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

import gym
env = gym.make('CartPole-v0')

In [2]:
# Constraints defining our neural network
learning_rate = 1e-1
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

X = tf.placeholder(tf.float32, [None, input_size], name="input_x")

# First layer of weights
W1 = tf.get_variable("W1", shape=[input_size, output_size],
                     initializer=tf.contrib.layers.xavier_initializer())
Qpred = tf.matmul(X, W1)

# We need to define the parts of the network needed for learning a policy
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

# Loss function
loss = tf.reduce_sum(tf.square(Y - Qpred))
# Learning
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Values for q learning
num_episodes = 2000
dis = 0.9
rList = []

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(num_episodes):
  e = 1. / ((i / 10) + 1)
  rAll = 0
  step_count = 0
  s = env.reset()
  done = False

  # The Q-Network training
  while not done:
    step_count += 1
    x = np.reshape(s, [1, input_size])
    # Choose an action by greedily (with e chance of random action) from the Q-network
    Qs = sess.run(Qpred, feed_dict={X: x})
    if np.random.rand(1) < e:
      a = env.action_space.sample()
    else:
      a = np.argmax(Qs)

    # Get new state and reward from environment
    s1, reward, done, _ = env.step(a)
    if done:
      Qs[0, a] = -100 # If the pole is dropped, -100 reward
    else:
      x1 = np.reshape(s1, [1, input_size])
      # Obtain the Q' values by feeding the new state through our network
      Qs1 = sess.run(Qpred, feed_dict={X: x1})
      Qs[0, a] = reward + dis * np.max(Qs1)

    # Train our network using target and predicted Q values on each episode
    sess.run(train, feed_dict={X: x, Y: Qs})
    s = s1
  
  rList.append(step_count)
  print("Episode: {} step: {}".format(i, step_count))
  # If last 10's avg steps are 500, it's good enough
  if len(rList) > 10 and np.mean(rList[-10:]) > 500:
    break

Episode: 0 step: 27
Episode: 1 step: 12
Episode: 2 step: 13
Episode: 3 step: 15
Episode: 4 step: 43
Episode: 5 step: 40
Episode: 6 step: 18
Episode: 7 step: 11
Episode: 8 step: 56
Episode: 9 step: 56
Episode: 10 step: 45
Episode: 11 step: 33
Episode: 12 step: 43
Episode: 13 step: 38
Episode: 14 step: 35
Episode: 15 step: 53
Episode: 16 step: 32
Episode: 17 step: 17
Episode: 18 step: 29
Episode: 19 step: 26
Episode: 20 step: 23
Episode: 21 step: 19
Episode: 22 step: 42
Episode: 23 step: 45
Episode: 24 step: 25
Episode: 25 step: 26
Episode: 26 step: 72
Episode: 27 step: 44
Episode: 28 step: 55
Episode: 29 step: 46
Episode: 30 step: 28
Episode: 31 step: 23
Episode: 32 step: 17
Episode: 33 step: 21
Episode: 34 step: 41
Episode: 35 step: 32
Episode: 36 step: 21
Episode: 37 step: 39
Episode: 38 step: 44
Episode: 39 step: 25
Episode: 40 step: 53
Episode: 41 step: 47
Episode: 42 step: 24
Episode: 43 step: 25
Episode: 44 step: 18
Episode: 45 step: 23
Episode: 46 step: 55
Episode: 47 step: 44
Ep

**It's very poor result, because**
1. the number of weights are too small
2. Correlation btw samples
3. Non-stationary targets